import tensorflow as tf from tensorflow.keras.applications import
ResNet50, ResNet101, ResNet152, ResNet34, ResNet18 from
tensorflow.keras.preprocessing.image import ImageDataGenerator from
tensorflow.keras.optimizers import Adam from tensorflow.keras.losses
import CategoricalCrossentropy from tensorflow.keras.callbacks import
EarlyStopping

tf.random.set_seed(42) 2. Pra-pemrosesan Data \# Direktori dataset
train_dir = ‘/path/to/imagenet/train’ val_dir = ‘/path/to/imagenet/val’

datagen = ImageDataGenerator( rescale=1.0/255, horizontal_flip=True,
rotation_range=20, zoom_range=0.2, shear_range=0.2, fill_mode=‘nearest’
)

train_gen = datagen.flow_from_directory( train_dir, target_size=(224,
224), batch_size=32, class_mode=‘categorical’ )

val_gen = datagen.flow_from_directory( val_dir, target_size=(224, 224),
batch_size=32, class_mode=‘categorical’ )

def create_model(base_model_class, num_classes=1000): \# Load
pre-trained ResNet model base_model =
base_model_class(weights=‘imagenet’, include_top=False,
input_shape=(224, 224, 3)) \# Add custom top layers x =
base_model.output x = tf.keras.layers.GlobalAveragePooling2D()(x) x =
tf.keras.layers.Dense(1024, activation=‘relu’)(x) predictions =
tf.keras.layers.Dense(num_classes, activation=‘softmax’)(x) model =
tf.keras.Model(inputs=base_model.input, outputs=predictions) \# Compile
model model.compile(optimizer=Adam(learning_rate=1e-4),
loss=CategoricalCrossentropy(), metrics=\[‘accuracy’\]) return model

def train_model(model, train_gen, val_gen, epochs=25): \# Callbacks
early_stopping = EarlyStopping(monitor=‘val_loss’, patience=3,
restore_best_weights=True)

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stopping] 
    )

    return history

# ResNet Variants

models = { ‘ResNet18’: ResNet18, ‘ResNet34’: ResNet34, ‘ResNet50’:
ResNet50, ‘ResNet101’: ResNet101, ‘ResNet152’: ResNet152 }

results = {}

for name, model_class in models.items(): print(f’Training {name}…’)
model = create_model(model_class) history = train_model(model,
train_gen, val_gen)

    # Evaluate model
    val_loss, val_accuracy = model.evaluate(val_gen)

    # Store results
    results[name] = {
        'val_accuracy': val_accuracy,
        'train_time': sum(history.history['time']),
        'num_parameters': model.count_params()
    }

    print(f'{name} - Validation Accuracy: {val_accuracy:.4f}')
    print(f'{name} - Training Time: {results[name]["train_time"]:.2f} hours')
    print(f'{name} - Number of Parameters: {results[name]["num_parameters"]:.2e}')

# Print a summary of results

for name, result in results.items(): print(f”{name}:“) print(f”
Validation Accuracy: {result\[‘val_accuracy’\]\*100:.2f}%“) print(f”
Training Time: {result\[‘train_time’\]:.2f} hours”) print(f” Number of
Parameters: {result\[‘num_parameters’\]:.2e}“) print()